# Preparing exporting QMOF data

In [ ]:
from mpcontribs.client import Client
import os
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import write

os.environ['MPCONTRIBS_API_KEY']='BEuiG4rxRKQR4M4q4S3tyHW7Y516a6Iz'
os.environ['SSL_CERT_FILE']='/Users/minhyukkang/opt/anaconda3/envs/TKenv/lib/python3.9/site-packages/certifi/cacert.pem'
client=Client(project='qmof')

# Downloading Files as .cif

In [ ]:
query_PBE = {'project':'qmof','formula__contains':'Cu', '_fields':['id','formula','data.EgPBE.value'], '_limit':500}
#subsitituting 'Cu' with another metal to get MOFs containing other metals
#modifying '_limit' to adjust the number of data MAX = 500
contribs_PBE = client.contributions.get_entries(**query_PBE).result()

contribs_PBE = contribs_PBE['data'].copy()

struct_ids = []
for contrib in contribs_PBE:
    c = client.get_contribution(contrib['id'])
    struct_ids.append(c["structures"][0]['id'])

#print(struct_ids)

ase_structs = []
for struct_id in struct_ids:
    s = client.get_structure(struct_id)
    s_ase = AseAtomsAdaptor.get_atoms(s)
    ase_structs.append(s_ase)

os.chdir('/Users/minhyukkang/Metal Structures')

for struct in ase_structs:
        write('geometry_{formula}.cif'.format(formula=struct.symbols), struct, wrap=True)

os.chdir('..')

# Calculating RDF and Find the minimum distance (Data Processing)

In [ ]:
#Creating PBE data named as 'data_PBE' using pandas dataframe
import pandas as pd

query_PBE = {'project':'qmof', 'formula_contains' : 'Cu', '_fields':['id','formula','data.EgPBE.value'], '_limit':500}
contribs_PBE = client.contributions.get_entries(**query_PBE).result()

contribs_PBE = contribs_PBE['data'].copy()

struct_ids = []
for contrib in contribs_PBE:
        c = client.get_contribution(contrib['id'])
        struct_ids.append(c["structures"][0]['id'])

#print(struct_ids)

ase_structs = []
for struct_id in struct_ids:
    s = client.get_structure(struct_id)
    s_ase = AseAtomsAdaptor.get_atoms(s)
    ase_structs.append(s_ase)

#print(ase_structs)

data_PBE = pd.DataFrame(data=contribs_PBE)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

print(data_PBE)

In [ ]:
#creating a list of metallic elements by sorting with electronegativity

from mendeleev import element
import mendeleev.fetch as m
ens = m.fetch_electronegativities()
mt = []
N = 1
for i in ens['Pauling']:
    if i < 2.5: mt.append(element(N).symbol)
    N = N + 1
non_mt = ['H','B','Si','P', 'Ge', 'As', 'Sb', 'Te', 'Po', 'At']
for j in non_mt:
    mt.remove(j)
print(mt)

In [ ]:
# RDF of a Crystal Structure

import numpy as np
from vasppy.rdf import RadialDistributionFunction
from pymatgen.core import Structure
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

r_zero=[] #for some molecules that has zero distance between metals after supercell
more_than_two_metals = []
excpt = []

#when using structures from .cif file
#for struct in ase_structs:
#    structure = Structure.from_file('/Users/minhyukkang/Metal_Structures/geometry_{formula}.cif'.format(formula=struct.symbols))


for i in mt:
    query_PBE = {'project':'qmof','formula__contains':i, '_fields':['id','formula','data.EgPBE.value'], '_limit':500}
    contribs_PBE = client.contributions.get_entries(**query_PBE).result()

    contribs_PBE = contribs_PBE['data'].copy()

    struct_ids = []
    r=[]
    n = 0

    for contrib in contribs_PBE:
            c = client.get_contribution(contrib['id'])
            struct_ids.append(c["structures"][0]['id'])

    for struct_id in struct_ids:
        structure = client.get_structure(struct_id)
        s_ase = AseAtomsAdaptor.get_atoms(structure)
        structure.make_supercell(2) #supercell by factor of 2 in all directions -- x,y,z
        #print(indices_cu)
        mt_for=[]
        PBE_value = float(str(pd.DataFrame(contribs_PBE)['data'][n]).replace('{\'EgPBE\': {\'value\': ','').replace('}}',''))
        n=n+1
        for x in mt:
            #getting indices of metal i and creating a list of metals in the specific formula -- mt_for
            if x in '{formula}'.format(formula=s_ase.symbols):
                exec(f'indices_{x} = [i for i, site in enumerate(structure) if site.species_string == x]')
                mt_for.append(x)
        try:
            if len(mt_for) == 1: #if there is only one metal -- metal itself
                y = mt_for[0]
                exec(f'rdf_{y}{y} = RadialDistributionFunction(structures=[structure],indices_i=indices_{y})')
                p = 0
                exec(f'if rdf_{y}{y}.r[rdf_{y}{y}.rdf.argmax()] > 0.1: p = find_peaks(rdf_{y}{y}.rdf)[0][0]')
                #finding the peak where rdf is the greatest
                if p==0: #molecules with errors
                    r_zero.append('{formula}'.format(formula=s_ase.symbols))
                    exec(f'min_r = rdf_{y}{y}.r[rdf_{y}{y}.rdf.argmax()]')
                else: #molecules without errors
                    exec(f'min_r = rdf_{y}{y}.r[p]')
                r.append(['{formula}'.format(formula=s_ase.symbols), min_r, PBE_value])
            elif len(mt_for) == 2: #for MOFs with two metals
                y0 = mt_for[0]
                y1 = mt_for[1]
                exec(f'rdf_{y0}{y1} = RadialDistributionFunction(structures=[structure],indices_i=indices_{y0},indices_j=indices_{y1})')
                exec(f'rdf_{y0}{y0} = RadialDistributionFunction(structures=[structure],indices_i=indices_{y0})')
                exec(f'rdf_{y1}{y1} = RadialDistributionFunction(structures=[structure],indices_i=indices_{y1})')
                exec(f'p_{y0}{y1}=0')
                exec(f'p_{y0}{y0}=0')     
                exec(f'p_{y1}{y1}=0')     
                exec(f'if rdf_{y0}{y1}.r[rdf_{y0}{y1}.rdf.argmax()] > 0.1: p_{y0}{y1} = find_peaks(rdf_{y0}{y1}.rdf)[0][0]')
                exec(f'if rdf_{y0}{y0}.r[rdf_{y0}{y0}.rdf.argmax()] > 0.1: p_{y0}{y0} = find_peaks(rdf_{y0}{y0}.rdf)[0][0]')
                exec(f'if rdf_{y1}{y1}.r[rdf_{y1}{y1}.rdf.argmax()] > 0.1: p_{y1}{y1} = find_peaks(rdf_{y1}{y1}.rdf)[0][0]')
                exec(f'min_r = min([rdf_{y0}{y1}.r[p_{y0}{y1}], rdf_{y0}{y0}.r[p_{y0}{y0}], rdf_{y1}{y1}.r[p_{y1}{y1}]])')
                r.append(['{formula}'.format(formula=s_ase.symbols), min_r, PBE_value])
            else:
                more_than_two_metals.append('{formula}'.format(formula=s_ase.symbols))

        except Exception:
            excpt.append('{formula}'.format(formula=s_ase.symbols))

    exec(f'data_combined_{i} = pd.DataFrame(data=r, columns = [\'Formula_{i}\', \'Radius\', \'PBE\'])')
    exec(f'print(data_combined_{i})')
    exec(f'plt.scatter(data_combined_{i}[\'Radius\'] , data_combined_{i}[\'PBE\'])')
    plt.xlabel('Radius')
    plt.ylabel('PBE')
    plt.title(i)
    plt.show()
    print(r_zero)
print(more_than_two_metals)

# Data Analysis

In [ ]:
#Drawing the RDF graph for a specific molecule

from pymatgen.core.structure import Structure
import numpy as np
from vasppy.rdf import RadialDistributionFunction
from pymatgen.core import Structure
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

A = ['Cu2C8Cl4H16N4O8']
#for i in list_small_r_Ha:
for i in A: 
    exec(f'structure = Structure.from_file(\'/Users/minhyukkang/Metal Structures/Cu_Structures/geometry_{i}.cif\')')
    structure.make_supercell(2)
    #print(indices_cu)
    #rdf_cucu = RadialDistributionFunction(structures=[structure],indices_i=indices_cu)
    mt_for=[]
    indices_Cu = [i for i, site in enumerate(structure) if site.species_string == 'Cu']
    for x in mt:
        if x in i:
            exec(f'indices_{x} = [i for i, site in enumerate(structure) if site.species_string == x]')
            mt_for.append(x)
    if len(mt_for) == 1:
        rdf_CuCu = RadialDistributionFunction(structures=[structure],indices_i=indices_Cu)
        max_rdf_CuCu = max(rdf_CuCu.rdf)
        p=0
        if rdf_CuCu.r[rdf_CuCu.rdf.argmax()] > 0.1:p = find_peaks(rdf_CuCu.rdf)[0][0]
        plt.plot(rdf_CuCu.r, rdf_CuCu.rdf, label = 'RDF')
        if p==0:
            r_zero.append('{formula}'.format(formula=struct.symbols))
            min_r = rdf_CuCu.r[rdf_CuCu.rdf.argmax()]
        else:
            min_r = rdf_CuCu.r[p]
        plt.scatter(min_r, 0, marker = '*', linewidths = 5, label = 'MIN_R')
        plt.legend()
        plt.xlabel('Radius')
        plt.ylabel('RDF value')
        plt.title(i)
        plt.show()
    elif len(mt_for) == 2:
        for y in mt_for:
            rdf_CuCu = RadialDistributionFunction(structures=[structure],indices_i=indices_Cu)
            exec(f'rdf_Cu{y} = RadialDistributionFunction(structures=[structure],indices_i=indices_Cu,indices_j=indices_{y})')
            exec(f'rdf_{y}{y} = RadialDistributionFunction(structures=[structure],indices_i=indices_{y})')
            p_CuCu = 0
            if rdf_CuCu.r[rdf_CuCu.rdf.argmax()] > 0.1: p_CuCu = find_peaks(rdf_CuCu.rdf)[0][0]
            exec(f'p_Cu{y}=0')
            exec(f'p_{y}{y}=0')              
            exec(f'if rdf_Cu{y}.r[rdf_Cu{y}.rdf.argmax()] > 0.1: p_Cu{y} = find_peaks(rdf_Cu{y}.rdf)[0][0]')
            exec(f'if rdf_{y}{y}.r[rdf_{y}{y}.rdf.argmax()] > 0.1: p_{y}{y} = find_peaks(rdf_{y}{y}.rdf)[0][0]')
            exec(f'min_r = min([rdf_CuCu.r[p_CuCu], rdf_Cu{y}.r[p_Cu{y}], rdf_{y}{y}.r[p_{y}{y}]])')
            plt.plot(rdf_CuCu.r, rdf_CuCu.rdf, label = 'rdf_CuCu')
            exec(f'plt.plot(rdf_Cu{y}.r, rdf_Cu{y}.rdf, label = \'rdf_Cu{y}\')')
            exec(f'plt.plot(rdf_{y}{y}.r, rdf_{y}{y}.rdf, label = \'rdf_{y}{y}\')')
            plt.scatter(min_r, 0, marker = '*', linewidths = 5, label = 'MIN_R')
            plt.legend()
            plt.xlabel('Radius')
            plt.ylabel('RDF value')
            plt.title(i)
            plt.show()
    else:
        more_than_two_metals.append('{formula}'.format(formula=struct.symbols))

In [ ]:
#Comparing MOFs with different metals

import matplotlib.pyplot as plt

plt.scatter(data_combined_Co['Radius'] , data_combined_Co['PBE'], label = 'Co')
plt.scatter(data_combined_Zn['Radius'] , data_combined_Zn['PBE'], label = 'Zn')
print(data_combined_Co.sort_values(by = 'Formula_Co'))
print(data_combined_Zn.sort_values(by = 'PBE'))
#plt.scatter(data_combined_r_list, data_combined_PBE_list)
#plt.yticks([0.6,0.8,1.0])
#plt.figure(figsize=(100, 100))
plt.legend()
plt.xlabel('Radius')
plt.ylabel('PBE')
plt.title('Radius vs PBE')
plt.show()

In [ ]:
#Finding molecules that have the same formula

Co_Zn_cmmn = []
n = 0
for i in data_combined_Co['Formula_Co']:
    a = i.replace('Co' ,'')
    m = 0
    for j in data_combined_Zn['Formula_Zn']:
        if str.__contains__(j, a) == True: 
                Co_Zn_cmmn.append([n, i, j,data_combined_Co['Radius'][n], data_combined_Zn['Radius'][m],  data_combined_Co['PBE'][n], data_combined_Zn['PBE'][m]])
        m = m+1
    n = n+1
pd.DataFrame(Co_Zn_cmmn, columns = ['n','Formula_Co', 'Formula_Zn','Radius_Co','Radius_Zn', 'PBE_Co', 'PBE_Zn'])

In [ ]:
cmmn = []
ni = 0
for i in data_combined_Co['Formula_Co']:
    a = i.replace('Co' ,'')
    nj = 0
    for j in data_combined_Ni['Formula_Ni']:
        if str.__contains__(j, a) == True:
            #print(i,j)
            nk = 0
            for k in data_combined_Zn['Formula_Zn']:
                if str.__contains__(k, a) == True:
                    cmmn.append([ni, i, j, k,  data_combined_Co['PBE'][ni], data_combined_Ni['PBE'][nj], data_combined_Zn['PBE'][nk]])
                    #nl = 0
                    #for l in data_combined_Zn['Formula_Zn']:
                    #    if str.__contains__(l, a) == True:
                    #        print(i,j,k,l)
                        #nl = nl+1
                nk = nk+1
        nj = nj+1
    ni = ni+1
pd_cmmn = pd.DataFrame(cmmn, columns = ['No.','Formula_Co', 'Formula_Ni','Formula_Zn' , 'PBE_Co', 'PBE_Ni', 'PBE_Zn'])
pd_cmmn

In [ ]:
#graphing a PBE trend for molecules shared by two metals

for i in range(15):
    plt.plot([1,2,3], [pd_cmmn['PBE_Co'][i], pd_cmmn['PBE_Ni'][i], pd_cmmn['PBE_Zn'][i]], label = pd_cmmn['Formula_Co'][i].replace('Co', 'Mt'))
plt.xticks([1,2,3], ['Co', 'Ni', 'Zn'])
plt.ylabel('Bandgap (eV) PBE')
plt.legend(bbox_to_anchor = (1.1,1.05)) 
plt.show()

In [ ]:
cmmn = []
ni = 0
for i in data_combined_Co['Formula_Co']:
    a = i.replace('Co' ,'')
    nj = 0
    for j in data_combined_Cu['Formula_Cu']:
        if str.__contains__(j, a) == True:
            #print(i,j)
            nk = 0
            for k in data_combined_Zn['Formula_Zn']:
                if str.__contains__(k, a) == True:
                    cmmn.append([i, j, k,data_combined_Co['PBE'][ni], data_combined_Cu['PBE'][nj], data_combined_Zn['PBE'][nk]])
                    #nl = 0
                    #for l in data_combined_Zn['Formula_Zn']:
                    #    if str.__contains__(l, a) == True:
                    #        print(i,j,k,l)
                        #nl = nl+1
                nk = nk+1
        nj = nj+1
    ni = ni+1
pd.DataFrame(cmmn, columns = ['Formula_Co', 'Formula_Cu','Formula_Zn', 'PBE_Co', 'PBE_Cu', 'PBE_Zn'])

In [ ]:
cmmn = []
ni = 0
for i in data_combined_Ni['Formula_Ni']:
    a = i.replace('Ni' ,'')
    nj = 0
    for j in data_combined_Cu['Formula_Cu']:
        if str.__contains__(j, a) == True:
            #print(i,j)
            nk = 0
            for k in data_combined_Zn['Formula_Zn']:
                if str.__contains__(k, a) == True:
                    cmmn.append([i, j, k,data_combined_Ni['Radius'][ni], data_combined_Cu['Radius'][nj], data_combined_Zn['Radius'][nk],  data_combined_Ni['PBE'][ni], data_combined_Cu['PBE'][nj], data_combined_Zn['PBE'][nk]])
                    #nl = 0
                    #for l in data_combined_Zn['Formula_Zn']:
                    #    if str.__contains__(l, a) == True:
                    #        print(i,j,k,l)
                        #nl = nl+1
                nk = nk+1
        nj = nj+1
    ni = ni+1
pd.DataFrame(cmmn, columns = ['Formula_Ni', 'Formula_Cu','Formula_Zn' ,'Radius_Ni','Radius_Cu','Radius_Zn', 'PBE_Ni', 'PBE_Cu', 'PBE_Zn'])

In [ ]:
#Drawing a box-and-whisker plot of Bandgaps for MOFs containing specific molecules

fig, ax = plt.subplots()
P4Mt = ['K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn'] 
for i in P4Mt:
    exec(f'{i}_PBE = data_combined_{i}[\'PBE\']')
ax.boxplot([K_PBE,Ca_PBE,Sc_PBE,Ti_PBE,V_PBE,Cr_PBE,Mn_PBE,Fe_PBE,Co_PBE,Ni_PBE,Cu_PBE,Zn_PBE])
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12], P4Mt)
plt.xlabel('Metals')
plt.ylabel('Bandgap (eV) PBE')
plt.title('Box-and-Whisker PBE')

In [ ]:
fig, ax = plt.subplots()
P5Mt = ['Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd'] 
P5Mt_list = []
for i in P5Mt:
    exec(f'{i}_PBE = data_combined_{i}[\'PBE\']')
ax.boxplot([Rb_PBE,Sr_PBE,Y_PBE,Zr_PBE,Nb_PBE,Mo_PBE,Tc_PBE,Ru_PBE,Rh_PBE,Pd_PBE,Ag_PBE,Cd_PBE])
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12], P5Mt)
plt.xlabel('Metals')
plt.ylabel('Bandgap (eV) PBE')
plt.title('Box-and-Whisker PBE')
print(P5Mt_list)

In [ ]:
#Creating PBE-only data
import pandas as pd
not_exist = []
cu = ['Cu']
for i in mt:
    query_PBE = {'project':'qmof','formula__contains':i, '_fields':['formula','data.EgPBE.value'], '_limit':500}
    contribs_PBE = client.contributions.get_entries(**query_PBE).result()

    contribs_PBE = contribs_PBE['data'].copy()
    pd_PBE = pd.DataFrame(contribs_PBE)
    PBE_list = []
    n = 0
    try:
        for j in pd_PBE['formula']:    
            PBE_list.append([j, float(str(pd_PBE['data'][n]).replace('{\'EgPBE\': {\'value\': ','').replace('}}',''))])
            n = n+1
    except:
        PBE_list.append([j, 0])
    exec(f'pd_PBE_{i} = pd.DataFrame(PBE_list, columns = [\'formula\', \'PBE\']).sort_values(by = \'PBE\')')
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)
    exec(f'print(pd_PBE_{i})')


In [ ]:
#Creating HSE-only data
for i in mt:
    query_HSE = {'project':'qmof','formula__contains':i, '_fields':['formula','data.EgHSE06.value'], '_limit':500}
    contribs_HSE = client.contributions.get_entries(**query_HSE).result()

    contribs_HSE = contribs_HSE['data'].copy()
    pd_HSE = pd.DataFrame(contribs_HSE)
    HSE_list = []
    n = 0
    try:
        for j in pd_HSE['formula']:    
            HSE_list.append([j, float(str(pd_HSE['data'][n]).replace('{\'EgHSE06\': {\'value\': ','').replace('}}',''))])
            n = n+1
    except:
        HSE_list.append([j, 0])
    exec(f'pd_HSE_{i} = pd.DataFrame(HSE_list, columns = [\'formula\', \'HSE\']).sort_values(by = \'HSE\')')
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)
    exec(f'print(pd_HSE_{i})')

In [ ]:
#Drawing density distribution graph for 

import seaborn as sns
import matplotlib.pyplot as plt

for i in mt:
    #exec(f'print(pd_PBE_{i})')
    #exec(f'print(pd_HSE_{i})')
    exec(f'sns.kdeplot(pd_PBE_{i}[\'PBE\'])')
    exec(f'sns.kdeplot(pd_HSE_{i}[\'HSE\'])')
    plt.legend(['PBE', 'HSE'])
    plt.title(i)
    plt.show()

In [ ]:
#Comparing Bandgap values using PBe and HSE

import matplotlib.pyplot as plt

plt.scatter(pd_PBE_Cu['PBE'], pd_PBE_Cu['HSE'])
plt.xlabel('PBE')
plt.ylabel('HSE')

In [ ]:
#Drawing r vs Bandgap(PBE) graph for molecules with low bandgap

list_data_combined = data_combined.values.tolist()
#print(list_data_combined)
list_low_PBE = []
for i in list_data_combined:
    if i[2] < 0.20:
        list_low_PBE.append(i)
for i in list_low_PBE:
    plt.scatter(i[1], i[2], c = 'b')
plt.xlim(2,10)
plt.xlabel('Radius')
plt.ylabel('PBE')
plt.title('Radius vs PBE (selected)')
plt.show()
print(pd.DataFrame(list_low_PBE, columns = ['formula', 'radius', 'PBE']).sort_values(by = ['PBE']))

In [ ]:
#Counting the number of molecules with low bandgap and contains halogen group

N_small_r = 0
list_small_r = []
Ha = ['F','Cl', 'Br', 'I']
N_small_r_Ha = 0
list_small_r_Ha = []
list_Ha = []
N_Ha = 0
for i in list_data_combined:
    if i[1] < 2.8:
        N_small_r = N_small_r + 1
        list_small_r.append([i[0],i[1]])
for j in list_small_r:
    for k in Ha:
        if k in j[0]:
            list_small_r_Ha.append(j[0])
            N_small_r_Ha = N_small_r_Ha + 1
for l in r:
    for m in Ha:
        if m in l[0]:
            list_Ha.append(l)
            N_Ha = N_Ha + 1
            
print(N_small_r)
print(N_small_r_Ha)
print(N_Ha)
print(pd.DataFrame(list_small_r, columns = ['Formula', 'Radius']).sort_values(by = 'Radius'))

In [ ]:
#Drawing r vs Bandgap(PBE) graph for molecules with in the selected range

list_trend_PBE = []
for i in list_data_combined:
    if i[1] > 4.0 and i[2] > 1.2 and i[2] < 2.645:
        list_trend_PBE.append(i)
pd_list_trend_PBE = pd.DataFrame(list_trend_PBE, columns = ['Formula','Radius', 'PBE']).sort_values(by = 'Radius')
print(pd_list_trend_PBE)
for i in list_trend_PBE:
    plt.scatter(i[1], i[2], c = 'b')
plt.xlabel('Radius')
plt.ylabel('PBE')
plt.title('Radius vs PBE (selected)')
plt.show()

In [ ]:
#linear regression line to analyse the trend line

import numpy as np 
import scipy

x1 = pd_list_trend_PBE['Radius']
y1 = pd_list_trend_PBE['PBE']

from sklearn.linear_model import LinearRegression

line_fitter = LinearRegression()
line_fitter.fit(x1.values.reshape(-1,1), y1)
r_squared = line_fitter.score(x1.values.reshape(-1,1), y1)
print([line_fitter.coef_, line_fitter.intercept_])
print(r_squared)
plt.plot(x1,line_fitter.predict(x1.values.reshape(-1,1)), label = 'Linear Regression')

#plt.plot(x2,y2, label = 'Logarithmic Regression')
for i in list_trend_PBE:
    plt.scatter(i[1], i[2], c = 'b')
plt.legend()
plt.xlabel('Radius')
plt.ylabel('PBE')
plt.title('Radius vs PBE (selected)')
plt.show()

In [ ]:
#Extending the linear regression line to explore molecules that share similar characteristics

list_trend_PBE = []
for i in list_data_combined:
    if i[2] < i[1]*0.27626993 + 0.21768791 + 0.2 and  i[2] > i[1]*0.27626993 + 0.21768791 - 0.2:
        list_trend_PBE.append(i)
pd_list_trend_PBE = pd.DataFrame(list_trend_PBE, columns = ['Formula','Radius', 'PBE']).sort_values(by = 'Radius')
print(pd_list_trend_PBE)
for i in list_trend_PBE:
    plt.scatter(i[1], i[2], c = 'b')
plt.xlabel('Radius')
plt.ylabel('PBE')
plt.title('Radius vs PBE (selected)')
plt.show()